## Variable and memory

- All variable in python are poiters, point to object in memory
- id() return the start position of memory allocation, an object can occupy more than one mem slot

In [1]:
import sys
import ctypes
import gc

In [2]:
# my_var and other_var both point to object 'Hello'
my_var = 'Hello'
other_var = my_var

In [3]:
# Python is pass by ref, pass my_var to function increase the reference by 1
print(sys.getrefcount(my_var))
# Use ctypes to avoid this or minus one from get ref
print(ctypes.c_long.from_address(id(my_var)).value)

other_var = 'Bye'
print(sys.getrefcount(my_var))
# Use ctypes to avoid this or minus one from get ref
print(ctypes.c_long.from_address(id(my_var)).value)
my_var = None
print(sys.getrefcount(my_var))
# Use ctypes to avoid this or minus one from get ref
print(ctypes.c_long.from_address(id(my_var)).value)

3
2
2
1
31325
31324


If object is None, we dont know what store in that address, num of ref become arbitary

Some value in python is pre-computed (often used value like 1,2,3,..100,'a','b',...) which make those reference initially large. This is call Python peephole.

In [4]:
a = 10000
b = 100
c = 1
d = 'a'
e = 'Hello'
g = [1,2,3]
print(sys.getrefcount(a))
print(ctypes.c_long.from_address(id(a)).value)
print(sys.getrefcount(b))
print(ctypes.c_long.from_address(id(b)).value)
print(sys.getrefcount(c))
print(ctypes.c_long.from_address(id(c)).value)
print(sys.getrefcount(d))
print(ctypes.c_long.from_address(id(d)).value)
print(sys.getrefcount(e))
print(ctypes.c_long.from_address(id(e)).value)
print(sys.getrefcount(g))
print(ctypes.c_long.from_address(id(g)).value)

3
2
121
120
2593
2592
286
285
3
2
2
1


### Garbage collection

When ref count hit zero, python will destroy object. But sometimes it not work. For example


Circular reference
- my_var ----> ObjA
- ObjA.val1 -----> ObjB
- ObjB.val2 -----> ObjA

Get rid of my_var will not auto destroy ObjA since ref count to A still 1 (point from ObjB), same as B.
This lead to memory leak.

Solve by garbage collector.
- control by gc module
- turned on by default
- could turn off but aware !!!

In [5]:
def ref_count(address):
    print(ctypes.c_long.from_address(address).value)

In [6]:
def object_by_id(object_id):
    for obj in gc.get_objects(): # object tracked by gc
        if id(obj) == object_id:
            return "Object exist"
    return "Not Found"

Circular ref

In [7]:
class A:
    def __init__(self) -> None:
        self.b = B(self)
        print(f"A: self {hex(id(self))} b: {hex(id(self.b))}")

class B:
    def __init__(self, a) -> None:
        self.a = a
        print(f"B: self {hex(id(self))} a: {hex(id(self.a))}")

In [8]:
gc.disable()

In [9]:
my_var = A()

B: self 0x1ce5536c4c0 a: 0x1ce5536cb50
A: self 0x1ce5536cb50 b: 0x1ce5536c4c0


In [10]:
a_id = id(my_var)
b_id = id(my_var.b)

In [11]:
ref_count(a_id)
ref_count(b_id)

2
1


In [12]:
object_by_id(a_id)

'Object exist'

In [13]:
object_by_id(b_id)

'Object exist'

In [14]:
my_var = None
ref_count(a_id)
ref_count(b_id)

1
1


In [15]:
object_by_id(a_id)

'Object exist'

In [16]:
object_by_id(b_id)

'Object exist'

In [17]:
gc.collect()

330

In [18]:
object_by_id(a_id)

'Not Found'

In [39]:
ref_count(a_id)
ref_count(b_id)

1
247269008


In [41]:
a = 10
b = 10
print(hex(id(a)))
print(hex(id(b)))
a = 15
print(hex(id(a)))
print(hex(id(b)))
a = a + 1
print(hex(id(a)))
print(hex(id(b)))

0x7ffc0ebd07d0
0x7ffc0ebd07d0
0x7ffc0ebd0870
0x7ffc0ebd07d0
0x7ffc0ebd0890
0x7ffc0ebd07d0
